In [1]:
from pyogrio import read_dataframe,write_dataframe
import geopandas as gpd
import os,glob,sys,time,re
import pandas as pd
import numpy as np
import multiprocessing as mp
from parallel_pandas import ParallelPandas
from tqdm.auto import tqdm
ParallelPandas.initialize(n_cpu=24, split_factor=16)

### get basin boundary files

In [ ]:
basins = glob.glob('../../data/GRIT/full_catchment/GRIT_full_catchment_*_EPSG8857_simplify_final.gpkg')
if not os.path.exists('../basin_boundary'):
    os.mkdir('../basin_boundary')
for basin in basins:
    gdf = read_dataframe(basin)
    
    # difference between ohdb_darea and grit_darea less than 20%
    gdf['bias'] = np.abs(gdf.grit_darea - gdf.ohdb_darea_hydrosheds) / gdf.ohdb_darea_hydrosheds * 100
    gdf1 = gdf.loc[gdf.bias<=20,:]
    
    # darea greater than 125 km2 to ensure at least one grid cell
    gdf1 = gdf1.loc[gdf1.grit_darea>=125,:]

    gdf1['segment_id'] = gdf1.segment_id.astype(int).astype(str)
    gdf1['reach_id'] = gdf1.segment_id.astype(int).astype(str)
    gdf1 = gdf1.rename(columns={'grit_darea':'gritDarea','ohdb_darea_hydrosheds':'ohdbDarea1','ohdb_darea':'ohdbDarea0'})
    
    # save
    basin1 = os.path.basename(basin)
    write_dataframe(gdf1, f'../basin_boundary/{basin1[:-5]}'+'_125km2.gpkg')
    gdf1 = gdf1.to_crs('epsg:4326')
    basin1 = re.sub('EPSG8857','EPSG4326',basin1)
    write_dataframe(gdf1, f'../basin_boundary/{basin1[:-5]}'+'_125km2.shp')
    print(basin)

### connect OHDB gauge with GLAKES polygons

In [ ]:
from pyogrio import read_dataframe
import geopandas as gpd
gdf_basin = read_dataframe('../basin_boundary/GRIT_full_catchment_all_EPSG8857_simplify_final_125km2_subset.gpkg')
df_attr = pd.read_csv('../data/basin_attributes.csv')
gdf_basin = gdf_basin.merge(df_attr[['ohdb_id','ohdb_longitude','ohdb_latitude']], on = 'ohdb_id')
p = gpd.GeoDataFrame(data = gdf_basin[['ohdb_id','ohdb_longitude','ohdb_latitude']], 
                     geometry = gpd.points_from_xy(gdf_basin.ohdb_longitude.values,gdf_basin.ohdb_latitude.values),
                     crs='epsg:4326')
p = p.to_crs('epsg:8857')
p['x_sta'] = p.geometry.centroid.x
p['y_sta'] = p.geometry.centroid.y
gdf_basin = gdf_basin.merge(p[['ohdb_id','x_sta','y_sta']], on = 'ohdb_id')

import glob
connect_all = []
for fname in glob.glob('../../data/geography/GLAKES/GLAKES/*shp'):
    gdf = read_dataframe(fname, columns = ['Lake_id','Area_bound','Lat','Lon'])
    p = gpd.GeoDataFrame(data = gdf[['Lake_id','Lat','Lon']], geometry = gpd.points_from_xy(gdf.Lon.values,gdf.Lat.values),crs='epsg:4326')
    p = p.to_crs('epsg:8857')
    p['x_res'] = p.geometry.centroid.x
    p['y_res'] = p.geometry.centroid.y
    gdf = gdf.to_crs('epsg:8857')
    gdf = gdf.merge(p[['Lake_id','x_res','y_res']], on = 'Lake_id')
    connect = gpd.sjoin(gdf_basin, gdf)
    connect = connect.groupby('ohdb_id').apply(
        lambda x:pd.Series([x.Lake_id.count(),
                            x.Area_bound.sum(),
                            np.min(np.sqrt((x.x_res-x.x_sta)**2+(x.y_res-x.y_sta)**2))/1000,
                            x.iloc[np.argmin((x.x_res-x.x_sta)**2+(x.y_res-x.y_sta)**2),:].Area_bound,
                           ], 
        index = ['num_lake','area_all_lakes','min_dis_km','area_nearest'])
    )
    connect_all.append(connect)
    print(fname)
connect_all = pd.concat(connect_all).reset_index()
connect_all = connect_all.groupby('ohdb_id').apply(
    lambda x:pd.Series([
        x.num_lake.sum(),
        x.area_all_lakes.sum(),
        x.min_dis_km.min(),
        x.iloc[np.argmin(x.min_dis_km),:].area_nearest,
    ], index = ['num_lake','area_all_lakes','min_dis_km','area_nearest'])
).reset_index()
connect_all.to_csv('../data/basin_connect_GLAKES.csv', index = False)

### select OHDB gauge and calculate streamflow indices

In [6]:
def cleanQ(df):
    # eliminate invalid records
    df1 = df.loc[df.Q.apply(lambda x: not isinstance(x, str)),:]
    df2 = df.loc[df.Q.apply(lambda x: isinstance(x, str)),:]
    try:
        df2 = df2.loc[df2.Q.str.match('\d+'),:]
    except:
        pass
    df = pd.concat([df1, df2])
    df['Q'] = df.Q.astype(np.float32)
    return df

def del_unreliableQ(df):
    '''
    all records are rounded to three decimal places
    observations less than 0 were flagged as suspected
    observations with more than ten consecutive equal values greater than 0 were flagged as suspected
    '''
    df = df.loc[df.Q>=0,:].reset_index()
    df['Q'] = df['Q'].round(3)
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').set_index('date')
    index = pd.date_range(df.index[0], df.index[-1], freq = 'D')
    df = df.reindex(index)
    df1 = df.diff()
    df1 = df1.where(df1==0, 1).diff()
    start = np.where(df1.values==-1)[0]
    end = np.where(df1.values==1)[0]
    if len(start) == 0 or len(end) == 0:
        # must no less than zero
        df = df.loc[df.Q>=0,:]
        return (df)
    if start[0] > end[0]:
        start = np.array([0]+start.tolist())
    if start[-1] > end[-1]:
        end = np.array(end.tolist()+[df1.shape[0]+10])
    duration = end - start
    start = start[duration>=10]
    end = end[duration>=10]
    del_idx = np.array([item for a,b in zip(start,end) for item in np.arange(a+1,b+2).tolist()])
    del_idx = del_idx[del_idx<df.shape[0]]
     # identical values should be greater than zero
    del_idx = [s for s in del_idx if df.iloc[s,:].Q > 0]
    if len(del_idx) > 0:
        df.drop(df.index[del_idx], inplace = True)
    # must no less than zero
    df = df.loc[df.Q>=0,:]
    return (df)

def del_outlierQ(df):
    '''
        Based on a previously suggested approach for evaluating temperature series (Klein Tank et al., 2009), 
        daily streamflow values are declared as outliers if values of log (Q+0.01) are larger or smaller than 
        the mean value of log (Q+0.01) plus or minus 6 times the standard deviation of log (Q+0.01) computed for 
        that calendar day for the entire length of the series. The mean and standard deviation are computed for 
        a 5-day window centred on the calendar day to ensure that a sufficient amount of data is considered. 
        The log-transformation is used to account for the skewness of the distribution of daily streamflow values 
        and 0.01 was added because the logarithm of zero is undefined. Outliers are flagged as suspect. 
        The rationale underlying this rule is that unusually large or small values are often associated with observational issues. 
        The 6 standard-deviation threshold is a compromise, aiming at screening out outliers that could come from 
        instrument malfunction, while not flagging extreme floods or low flows.
    '''
    df['logQ'] = np.log(df['Q']+0.01)
    df['doy'] = df.index.dayofyear
    df['year'] = df.index.year
    df = df.pivot_table(index = 'doy', columns = 'year', values = 'logQ').reset_index()
    def tmp(x0):
        x = np.arange(x0-2, x0+3) 
        x = np.where(x <= 0, x + 366, x)
        x = np.where(x > 366, x - 366, x)
        s = df.loc[df.doy.isin(x),:].drop(columns=['doy']).values.flatten()
        ave = np.nanmean(s)
        std = np.nanstd(s)
        low = ave - std * 6
        upp = ave + std * 6
        return (x0, low, upp)
    thres = list(map(tmp, np.arange(1, 367)))
    thres = pd.DataFrame(data = np.array(thres), columns = ['doy','low','upp'])
    df = df.merge(thres, on = 'doy').set_index('doy')
    df.iloc[:,:(df.shape[1]-2)] = df.iloc[:,:(df.shape[1]-2)].where(df.iloc[:,:(df.shape[1]-2)].lt(df['upp'], axis=0))
    df.iloc[:,:(df.shape[1]-2)] = df.iloc[:,:(df.shape[1]-2)].where(df.iloc[:,:(df.shape[1]-2)].gt(df['low'], axis=0))
    df = df.drop(columns = ['low','upp']).stack().reset_index(name='logQ')
    df['Q'] = np.exp(df['logQ']) - 0.01
    df['Q'] = np.where(df['Q'].abs()<1e-6, 0, df['Q'])
    df['date'] = pd.to_datetime(df['level_1'].astype(str) + '-' + df['doy'].astype(str), format='%Y-%j')
    df = df[['date','Q']].sort_values('date').set_index('date')
    return df

def remove_periodic_zero(df):
    df['year'] = df.index.year
    df['doy'] = df.index.dayofyear
    df1 = df.pivot_table(index = 'year', columns = 'doy', values = 'Q').fillna(0)
    s = (df1==0).all()
    s.name = 'periodic'
    df = df.merge(pd.DataFrame(s).reset_index(), on = 'doy')
    df.loc[df.periodic==True,'Q'] = np.nan
    df['date'] = pd.to_datetime(df['year'] * 1000 + df['doy'], format='%Y%j')
    df = df[['date','Q']].set_index('date')
    return df

def main(par, scale = 'season'):
    ohdb_id, Darea = par
    df = pd.read_csv(os.environ['DATA']+f'/data/OHDB/OHDB_v0.2.3/OHDB_data/discharge/daily/{ohdb_id}.csv')
    # read
    df = cleanQ(df)
    # quality check
    df = del_unreliableQ(df)
    # delete outliers
    df = del_outlierQ(df)
    # periodic zero not included
    df = remove_periodic_zero(df)
#     # only retain records with at least 328 observations (90%) are required
#     tmp = df.resample('YE')['Q'].agg(countDay = lambda x:x.shape[0])
#     if tmp.loc[tmp.countDay>=328,:].shape[0] == 0:
#         return
#     years = tmp.loc[(tmp.countDay>=328)&(tmp.index.year>=1982),:].index.year.tolist()
#     if tmp.loc[(tmp.countDay>=300)&(tmp.index.year==2023),:].shape[0] > 0:
#         years = years + [2023]
    df = df.loc[(df.index.year>=1981)&(df.index.year<=2023),:]
    years = df.index.year.unique()
    # only retain gauge with at least 20 years of AMS during 1982-2023
    if len(years[years>=1982]) < 20:
        return
    df = df.loc[df.index.year.isin(years),:]
    # filter gauge with coefficient of variability less than 0.05
    if df.shape[0] == 0 or df.Q.std() / df.Q.mean() < 0.05:
        return
    
    # reindex
    newindex = pd.date_range(df.index.values[0], df.index.values[-1], freq = 'D')
    df = df.reindex(newindex)
    
    # 7-day moving average
    df['Q7'] = df.rolling(7).mean().dropna()
    df['year'] = df.index.year
    
    df['winter_year'] = df.index.year
    df.loc[df.index.month == 12, 'winter_year'] += 1
    
    df['season'] = 'DJF'
    df.loc[(df.index.month>=3)&(df.index.month<=5),'season'] = 'MAM'
    df.loc[(df.index.month>=6)&(df.index.month<=8),'season'] = 'JJA'
    df.loc[(df.index.month>=9)&(df.index.month<=11),'season'] = 'SON'
    df = df.dropna()

    #     df['season'] = '11-4'
    #     df.loc[(df.index.month>=5)&(df.index.month<=10),'season'] = '5-10'
    if scale == 'year':
        # count observations and calculate Qmax7 and Qmin7 for each year
        df1 = df.groupby('year').agg(countDay = ('Q',lambda x:x.shape[0]), 
                                        Qmax7 = ('Q7',lambda x:x.max()),
                                        Qmax = ('Q',lambda x:x.max()),
                                        Qmin7 = ('Q7',lambda x:x.min()),
                                        Q7zeroCount = ('Q7',lambda x:np.sum(x==0)),
                                        Qmaxdate = ('Q',lambda x:x.idxmax()),
                                        Qmax7date = ('Q7',lambda x:x.idxmax()),
                                        Qmin7date = ('Q7',lambda x:x.idxmin()),
                                        )
    elif scale == 'season':
        # count observations and calculate Qmax7 and Qmin7 for each season
        df1 = df.groupby([scale,'winter_year']).agg(countDay = ('Q',lambda x:x.shape[0]), 
                                        Qmax7 = ('Q7',lambda x:x.max()),
                                        Qmax = ('Q',lambda x:x.max()),
                                        Qmin7 = ('Q7',lambda x:x.min()),
                                        Q7zeroCount = ('Q7',lambda x:np.sum(x==0)),
                                        Qmaxdate = ('Q',lambda x:x.idxmax()),
                                        Qmax7date = ('Q7',lambda x:x.idxmax()),
                                        Qmin7date = ('Q7',lambda x:x.idxmin()),
                                        )
        df1 = df1.loc[df1.countDay>=80,:] # at least 80 days of records to calculate seasonal extremes
#         df1 = df1.loc[df1.countDay>=150,:] # at least 150 days of records to calculate double-seasonal extremes
    else:
        raise Exception('scale must be season or year')
    df1['Qmax7date'] = pd.to_datetime(df1['Qmax7date'])
    df1['Qmin7date'] = pd.to_datetime(df1['Qmin7date'])
    df1['Qmaxdate'] = pd.to_datetime(df1['Qmaxdate'])
    
    # Q must not be lower than zero
    df1['Qmax7'] = df1.Qmax7.where(df1.Qmax7>=0, np.nan)
    df1['Qmin7'] = df1.Qmin7.where(df1.Qmin7>=0, np.nan)
    df1['Qmax'] = df1.Qmax.where(df1.Qmax>=0, np.nan)

    if df1.shape[0] == 0:
        return
    df1 = df1.reset_index()
    df1['ohdb_id'] = ohdb_id
    return (df1)

if __name__ == '__main__':
    # if not os.path.exists('../data/OHDB_metadata_subset.csv'):
    #     # select gauges that have good basin boundary
    #     df = pd.read_csv('../../data/OHDB/OHDB_v0.2.3/OHDB_metadata/OHDB_metadata.csv')
    #     df1 = []
    #     for fname in glob.glob('../basin_boundary/GRIT*8857*'):
    #         gdf = read_dataframe(fname, read_geometry = False)
    #         print(gdf.shape, gdf.ohdb_id.unique().shape)
    #         df1.append(df.loc[df.ohdb_id.isin(gdf.ohdb_id.unique()),:])
    #     df1 = pd.concat(df1)
    #     df1.to_csv('../OHDB_metadata_subset.csv', index = False)
    # else:
    gdf = read_dataframe('../basin_boundary/GRIT_full_catchment_all_EPSG8857_simplify_final_125km2_subset.gpkg')
    print(gdf.shape)
    ohdb_ids = gdf.ohdb_id.values
    pool = mp.Pool(48)
    pars = gdf[['ohdb_id','gritDarea']].values.tolist()
    df = pool.map(main, pars)
    print('Finish processing, then start merging')
    df = pd.concat(df)
    print('Finish merging', df.shape)
    df.to_csv('../data/dis_OHDB_seasonal4_Qmin7_Qmax7_1982-2023.csv', index = False)
    print(df.Qmin7.isna().sum(), df.Qmax7.isna().sum(), df.ohdb_id.unique().shape)
    

(10717, 9)
Finish processing, then start merging
Finish merging (1536373, 11)
0 0 (10586,)


### transform MSWX meteo files to seperate files for each gauge

In [ ]:
from pyogrio import read_dataframe,write_dataframe
import geopandas as gpd
import os,glob,sys,time,re
import pandas as pd
import numpy as np
import multiprocessing as mp
from parallel_pandas import ParallelPandas
from tqdm.auto import tqdm
ParallelPandas.initialize(n_cpu=16, split_factor=16)

# transform all the meteo files into separate csv files for each gauge
def read(year, meteo = 'MSWX'):
    if meteo == 'ERA5':
        fname = f'../ee_era5_land/ERA5_Land_daily_meteorology_for_OHDB_10717_stations_{year}.csv'
    elif meteo == 'MSWX':
        fname = f'../data_mswx/MSWX_daily_meteorology_for_OHDB_10717_stations_{year}.csv'
    df = pd.read_csv(fname).set_index('ohdb_id')
    print(year)
    return df
pool = mp.Pool(8)
df_meteo = pool.map(read, np.arange(1982, 2024).tolist())
df_meteo = pd.concat(df_meteo, axis = 1)
df_meteo = df_meteo.rename(columns=lambda x:x.lower())
df_meteo = df_meteo.round(6)
df_meteo.loc[:,df_meteo.columns.str.endswith(('_p','_tmax','_tmin','_wind'))] = df_meteo.loc[:,df_meteo.columns.str.endswith(('_p','_tmax','_tmin','_wind'))].round(2)
df_meteo = df_meteo.reset_index()

def func_main(x, meteo = 'MSWX'):
    ohdb_id = x.ohdb_id
    if os.path.exists(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv'):
        try:
            a = pd.read_csv(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv')
            if a.shape[0] < 15300:
                os.remove(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv')
            else:
                return
        except:
            os.remove(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv')
    x = x.drop(index=['ohdb_id'])
    x.name = 'value'
    y = []
    for name in ['p','tmax','tmin','lwd','pres','relhum','spechum','swd','wind']:
        y0 = x.loc[x.index.str.endswith('_'+name)]
        y0.name = name
        y0.index = y0.index.str[:8]
        y.append(y0)
    y = pd.concat(y, axis = 1)
    # x = x.pivot_table(index = 'date', columns = 'meteo', values = 'value').rename(columns=lambda x:x.lower()).reset_index()
    # x['date'] = pd.to_datetime(x.date.values)
    # x.to_csv(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv', index = False)
    y['date'] = pd.to_datetime(y.index.values, format = '%Y%m%d')
    y.to_csv(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv', index = False)

df_meteo.p_apply(func_main, axis = 1)

### transform MSWEP and GLEAM files to separate files for each gauge

In [ ]:
from pyogrio import read_dataframe,write_dataframe
import geopandas as gpd
import os,glob,sys,time,re
import pandas as pd
import numpy as np
import multiprocessing as mp
from parallel_pandas import ParallelPandas
from tqdm.auto import tqdm
ParallelPandas.initialize(n_cpu=16, split_factor=16)

# transform all the meteo files into separate csv files for each gauge
def read(par):
    year, meteo = par
    if meteo == 'MSWEP':
        df = pd.read_csv(f'../data/GRIT_catch_ave_pr_MSWEP_{year}.csv').set_index('ohdb_id')
    elif meteo == 'E':
        df = pd.read_csv(f'../gleam_data/E_{year}_GLEAM_v4.1a.csv').set_index('ohdb_id')
    elif meteo == 'SM':
        df = pd.read_csv(f'../gleam_data/SMrz_{year}_GLEAM_v4.1a.csv').set_index('ohdb_id')
    if meteo == 'MSWEP':
        df = df / 10
    print(year)
    return df
pool = mp.Pool(8)
df_mswep = pool.map(read, [(a,'MSWEP') for a in np.arange(1982, 2024).tolist()])
df_mswep = pd.concat(df_mswep, axis = 1)

df_e = pool.map(read, [(a,'E') for a in np.arange(1982, 2024).tolist()])
df_e = pd.concat(df_e, axis = 1)

df_sm = pool.map(read, [(a,'SM') for a in np.arange(1982, 2024).tolist()])
df_sm = pd.concat(df_sm, axis = 1)

In [ ]:
def func_main(x, meteo = 'MSWX'):
    ohdb_id = x.ohdb_id
    if os.path.exists(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv'):
        try:
            a = pd.read_csv(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv')
            if a.shape[0] < 15300:
                os.remove(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv')
            else:
                return
        except:
            os.remove(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv')
    x = x.drop(index=['ohdb_id'])
    x.name = 'value'
    y = []
    for name in ['p','tmax','tmin','lwd','pres','relhum','spechum','swd','wind']:
        y0 = x.loc[x.index.str.endswith('_'+name)]
        y0.name = name
        y0.index = y0.index.str[:8]
        y.append(y0)
    y = pd.concat(y, axis = 1)
    # x = x.pivot_table(index = 'date', columns = 'meteo', values = 'value').rename(columns=lambda x:x.lower()).reset_index()
    # x['date'] = pd.to_datetime(x.date.values)
    # x.to_csv(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv', index = False)
    y['date'] = pd.to_datetime(y.index.values, format = '%Y%m%d')
    y.to_csv(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv', index = False)

df_meteo.p_apply(func_main, axis = 1)

### connect MSWX with snowmelt and snowfall

In [ ]:
# def snow_to_each_file(x):
#     ohdb_id = x.name
# #     tmp = pd.read_csv(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv')
#     x = x.reset_index().rename(columns={'time':'date',ohdb_id:'snowmelt'})
# #     y = tmp.merge(x, on = 'date')
#     x.to_csv(f'../data_mswx/mswx_each_basin/{ohdb_id}_snow.csv', index = False)
# for name in ['snow']:
#     fnames = glob.glob(f'../data_mswx/GRIT_catch_ave_{name}_[0-9]*csv')
#     df = pd.concat([pd.read_csv(fname).set_index('time') for fname in fnames], axis = 0)
#     df.p_agg(snow_to_each_file)

import multiprocessing as mp
def func_merge_snow(ohdb_id):
    df = pd.read_csv(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv')
    snowfall = pd.read_csv(f'../data_mswx/mswx_each_basin/{ohdb_id}_snowfall.csv')
    snowmelt = pd.read_csv(f'../data_mswx/mswx_each_basin/{ohdb_id}_snowmelt.csv')
    snow = pd.read_csv(f'../data_mswx/mswx_each_basin/{ohdb_id}_snow.csv')
    df = df.merge(snowfall.rename(columns={'snowmelt':'snowfall'}), on = 'date')
    df = df.merge(snowmelt, on = 'date')
    df = df.merge(snow.rename(columns={'snowmelt':'snow'}), on = 'date')
    df.to_csv(f'../data_mswx/mswx_each_basin/{ohdb_id}_new.csv', index = False)
    print(ohdb_id)

import glob,os
ohdb_ids = [os.path.basename(fname).split('.')[0] for fname in glob.glob('../data_mswx/mswx_each_basin/OHDB_*[0-9].csv')]
print(len(ohdb_ids))
pool = mp.Pool(16)
pool.map(func_merge_snow, ohdb_ids)

### connect Qmax7 and Qmin7 with meteo

In [2]:
def func_meteo(x, name = 'Qmax7date'):
    ohdb_id = x.xxx.values[0]
    x = x.drop(columns=['xxx'])
    df_meteo = pd.read_csv(f'../data_mswx/mswx_each_basin/{ohdb_id}_new.csv')
    df_meteo['date'] = pd.to_datetime(df_meteo['date'])
    df_meteo['ohdb_id'] = ohdb_id
    x = x[['ohdb_id',name]].merge(df_meteo,on = 'ohdb_id')
    x['tmp'] = (x.date - x[name]).dt.days
    x3 = x.loc[(x.tmp>-3)&(x.tmp<=0),:].drop(columns=['date','ohdb_id','tmp']).groupby(name).mean().rename(columns=lambda x:x+'_3')
    x7 = x.loc[(x.tmp>-7)&(x.tmp<=0),:].drop(columns=['date','ohdb_id','tmp']).groupby(name).mean().rename(columns=lambda x:x+'_7')
    x15 = x.loc[(x.tmp>-15)&(x.tmp<=0),:].drop(columns=['date','ohdb_id','tmp']).groupby(name).mean().rename(columns=lambda x:x+'_15')
    x30 = x.loc[(x.tmp>-30)&(x.tmp<=0),:].drop(columns=['date','ohdb_id','tmp']).groupby(name).mean().rename(columns=lambda x:x+'_30')
    x365 = x.loc[(x.tmp>-365)&(x.tmp<=0),:].drop(columns=['date','ohdb_id','tmp']).groupby(name).mean().rename(columns=lambda x:x+'_365')
    x = pd.concat([x3,x7,x15,x30,x365], axis = 1).reset_index()
    return x

df_flood = pd.read_csv('../data/dis_OHDB_seasonal4_Qmin7_Qmax7_1982-2023.csv')
df_flood['Qmax7date'] = pd.to_datetime(df_flood['Qmax7date'])
df_flood['Qmin7date'] = pd.to_datetime(df_flood['Qmin7date'])
df_flood['Qmaxdate'] = pd.to_datetime(df_flood['Qmaxdate'])

df_flood['xxx'] = df_flood['ohdb_id'].values

df2 = df_flood.groupby('ohdb_id').p_apply(lambda x: func_meteo(x, name = 'Qmax7date')).reset_index().drop(columns = ['level_1'])
df2.to_csv('../data/Qmax7_seasonal4_multi_MSWX_meteo.csv', index = False)
del df2

df2 = df_flood.groupby('ohdb_id').p_apply(lambda x: func_meteo(x, name = 'Qmin7date')).reset_index().drop(columns = ['level_1'])
df2.to_csv('../data/Qmin7_seasonal4_multi_MSWX_meteo.csv', index = False)
del df2

df2 = df_flood.groupby('ohdb_id').p_apply(lambda x: func_meteo(x, name = 'Qmaxdate')).reset_index().drop(columns = ['level_1'])
df2.to_csv('../data/Qmax_seasonal4_multi_MSWX_meteo.csv', index = False)

<LAMBDA> DONE:   0%|          | 0/10520 [00:00<?, ?it/s]

<LAMBDA> DONE:   0%|          | 0/10520 [00:00<?, ?it/s]

<LAMBDA> DONE:   0%|          | 0/10520 [00:00<?, ?it/s]

### subset basin boundary files again

In [ ]:
df = pd.read_csv('../dis_OHDB_Qmin7_Qmax7_1982-2023.csv')
ohdb_ids = df.ohdb_id.unique()
for fname in glob.glob('../basin_boundary/GRIT*8857*'):
    gdf = read_dataframe(fname)
    gdf = gdf.loc[gdf.ohdb_id.isin(ohdb_ids),:]
    write_dataframe(gdf, fname[:-5]+'_subset.gpkg')
    print(fname)

### use catch_mean_GLHYMPS_GLiM.py to get catchment average subsurface characteristics

### calculate number of upstream dams

In [ ]:
gdf_dam = read_dataframe('../../data/geography/GDAT_data_v1/data/GDAT_v1_dams.shp')
gdf_dam = gdf_dam.to_crs('espg:8857')
gdf = read_dataframe('../basin_boundary/GRIT_full_catchment_all_EPSG8857_simplify_final_125km2_subset.gpkg')
join = gpd.overlay(gdf_dam, gdf)
join = join.groupby('ohdb_id')['Feature_ID'].count().rename(columns={'Feature_ID':'dam_num'})
join = join.reindex(gdf.ohdb_id.values).fillna(0).reset_index()
join.to_csv('../geography/dam_num.csv', index = False)

### extract average meteorological conditions in the past 7 days preceding Qmax7 and Qmin7

### merge basin attributes

In [25]:
import pandas as pd
import numpy as np
import glob,os

fnames = glob.glob('../geography/*csv')
df_all = []
for fname in fnames:
    df = pd.read_csv(fname)
    if df.shape[1] >= 10717:
        df = df.T.reset_index()
    if df.shape[0] >= 10717:
        df = df
    name = os.path.basename(fname).split('_')[0]
    try:
        df = df.loc[df['index']!='time',:]
    except:
        df = df.loc[df.ohdb_id!='time',:]
    df = df.rename(columns={'index':'ohdb_id'}).set_index('ohdb_id')
    if '0-5cm' in fname:
        name = name + '_layer1'
    elif '5-15cm' in fname:
        name = name + '_layer2'
    elif '15-30cm' in fname:
        name = name + '_layer3'
    elif '30-60cm' in fname:
        name = name + '_layer4'
    elif '60-100cm' in fname:
        name = name + '_layer5'
    elif '100-200cm' in fname:
        name = name + '_layer6'
    if df.shape[1] == 1:
        df.columns = [name]
    if 'LAI' in fname:
        df = df.iloc[:-1,:2]
        df.columns = ['LAI','FAPAR']
    df_all.append(df)
df_all = pd.concat(df_all, axis = 1)
df_all = df_all.reset_index().rename(columns={'index':'ohdb_id'})

# merge metadata
df_meta = pd.read_csv('../data/OHDB_metadata_subset.csv')
df_all = df_all.merge(df_meta, on = 'ohdb_id')

df_all.to_csv('../data/basin_attributes_new.csv', index = False)
df_all.shape

(10717, 60)

### use GDAT reservoir area / catchment area to indicate the impact of reservoir regulation

In [ ]:
gdf_res = read_dataframe('../../data/geography/GDAT_data_v1/data/GDAT_v1_catchments.shp').to_crs('epsg:8857')
gdf_dam = read_dataframe('../../data/geography/GDAT_data_v1/data/GDAT_v1_dams.shp').to_crs('epsg:8857')
gdf_basin = read_dataframe('../basin_boundary/GRIT_full_catchment_all_EPSG8857_simplify_final_125km2_subset.gpkg')

In [ ]:
inter = gpd.sjoin(gdf_basin, gdf_dam)
gdf_res['darea'] = gdf_res.area / 1000000
inter = inter.merge(gdf_res[['Feature_ID','Dam_Name','darea']], on = ['Feature_ID','Dam_Name'])
inter.loc[inter.Year_Fin=='BLANK','Year_Fin'] = None
inter['Year_Fin'] = pd.to_numeric(inter['Year_Fin'])

inter.loc[inter.Year_Const.isna(),'Year_Const'] = np.nan
inter.loc[~inter.Year_Const.isna(),'Year_Const'] = inter.loc[~inter.Year_Const.isna(),'Year_Const'].str[:4].astype(int)
inter['year'] = inter[['Year_Fin','Year_Const']].min(axis=1)
inter = inter.groupby(['ohdb_id','gritDarea']).apply(lambda x:pd.Series([
    x.darea.sum(),
    x.year.mean(),
    x.Main_P_Map.mode().values[0] if len(x.Main_P_Map.mode().values)>=1 else 'Hydroelectricity'
], index = ['res_darea_normalize','Year_ave','Main_Purpose_mode'])).reset_index()
inter['Year_ave'] = inter['Year_ave'].fillna(2000)
inter['Main_Purpose_mode'] = inter['Main_Purpose_mode'].fillna('Hydroelectricity')
inter['res_darea_normalize'] = inter.res_darea_normalize / inter.gritDarea
inter.to_csv('../data/dam_impact.csv', index = False)
print(inter.head())

### calculate annual average and std of Tmax and Tmin as basin attributes

In [ ]:
import pandas as pd
import glob
fnames = glob.glob('../data_mswx/*meteo*')
def func(fname):
    df = pd.read_csv(fname)
    df = df.set_index('ohdb_id')
    df = df.loc[:,(df.columns.str.contains('Tmax'))|(df.columns.str.contains('Tmin'))]
    return (df)
import multiprocessing as mp
pool = mp.Pool(8)
df = pool.map(func, fnames)
df = pd.concat(df, axis = 1)
df_Tmax = pd.concat([
    df.loc[:,df.columns.str.contains('Tmax')].mean(axis = 1),
    df.loc[:,df.columns.str.contains('Tmax')].std(axis = 1)
], axis = 1)
df_Tmin = pd.concat([
    df.loc[:,df.columns.str.contains('Tmin')].mean(axis = 1),
    df.loc[:,df.columns.str.contains('Tmin')].std(axis = 1)
], axis = 1)

df_Tmax.columns = ['tmax_ave','tmax_std']
df_Tmin.columns = ['tmin_ave','tmin_std']
df_Tmax.head()

In [ ]:
df_Tmax.columns = ['tmax_ave','tmax_std']
df_Tmin.columns = ['tmin_ave','tmin_std']
df_attr = pd.read_csv('../data/basin_attributes.csv').set_index('ohdb_id')
df_attr = df_attr.loc[:,~df_attr.columns.str.contains('tmax')]
df_attr = df_attr.loc[:,~df_attr.columns.str.contains('tmin')]
df_attr = pd.concat([df_Tmax, df_attr, df_Tmin], axis = 1).reset_index()
df_attr.to_csv('../data/basin_attributes.csv', index = False)

df_attr.head()

In [16]:
import pandas as pd
target = 'Qmax7'
# read meteo dataset
fname = f'../data/{target}_seasonal4_multi_MSWX_meteo.csv'
df_meteo = pd.read_csv(fname)

# merge with discharge
df_dis = pd.read_csv('../data/dis_OHDB_seasonal4_Qmin7_Qmax7_1982-2023.csv')

df_meteo[target+'date'] = pd.to_datetime(df_meteo[target+'date'])
df_dis[target+'date'] = pd.to_datetime(df_dis[target+'date'])

df_meteo = df_meteo.merge(df_dis[[target+'date','season','year',target, 'ohdb_id']], on = ['ohdb_id', target+'date'])
df_meteo = df_meteo.rename(columns={target:'Q'})

df_meteo.head()

,ohdb_id,Qmax7date,p_3,tmax_3,tmin_3,lwd_3,pres_3,relhum_3,spechum_3,swd_3,...,relhum_365,spechum_365,swd_365,wind_365,snowfall_365,snowmelt_365,snow_365,season,year,Q
0,OHDB_001000001,1982-02-23,12.130000,29.746667,22.396667,416.689829,97624.539062,89.719617,0.018836,182.427806,...,88.309941,0.018450,193.707798,1.033889,0.0,0.0,0.0,DJF,1982,141.059424
1,OHDB_001000001,1982-04-14,9.686667,29.323333,22.413333,412.342183,97680.229167,89.839584,0.018798,175.331294,...,88.365117,0.018500,194.285508,0.998365,0.0,0.0,0.0,MAM,1982,94.455134
2,OHDB_001000001,1982-06-01,0.316667,30.146667,19.896667,390.269114,98216.557292,82.823423,0.016004,197.838913,...,88.407879,0.018295,189.422901,0.999079,0.0,0.0,0.0,JJA,1982,16.824143
3,OHDB_001000001,1982-11-28,10.410000,30.016667,22.560000,414.350128,97579.018229,86.173724,0.018418,166.310639,...,88.696003,0.018305,194.952964,0.921536,0.0,0.0,0.0,SON,1982,96.043722
4,OHDB_001000001,1983-02-25,8.410000,29.716667,23.350000,424.421173,97658.708333,89.706668,0.019485,193.830856,...,88.290538,0.018455,198.201429,0.936685,0.0,0.0,0.0,DJF,1983,76.552865


### connect gauge to Koppen 2nd climate classification

In [11]:
import pandas as pd
import numpy as np
df = pd.read_table('../../data/geography/koppen_map_1980-2099/legend.txt', skiprows = 3, nrows = 30, header = None)
df.columns = ['tmp']
df['koppen_code'] = df.tmp.apply(lambda x:int(x.split(':')[0]))
df['koppen_label'] = df.tmp.apply(lambda x:x.split(':')[1].split()[0])

df['koppen_text1'] = df.tmp.apply(lambda x:x.split(':')[1].split(',')[0].split()[1])
df['koppen_text2'] = df.tmp.apply(lambda x:x.split(':')[1].split(',')[1])
df['koppen_text2'] = df['koppen_text2'].apply(lambda x:x.split()[0].strip() if '[' in x else x)
df['koppen_text3'] = df.tmp.apply(lambda x:x.split(':')[1].split(',')[-1])
df['koppen_text3'] = df['koppen_text3'].str.extract(r"([^\d]+)")
df['koppen_text3'] = df['koppen_text3'].apply(lambda x:x[:-1].strip())
df['koppen_text3'] = np.where(df['koppen_text3'] == df['koppen_text2'], None, df['koppen_text3'])
df = df.drop(columns=['tmp'])

import xarray as xr
ds = xr.open_dataset('../../data/geography/koppen_map_1980-2099/1991_2020/koppen_geiger_0p01.nc')
df1 = pd.read_csv('../data/basin_attributes.csv')
koppen = []
for ohdb_id0,lon0,lat0 in zip(df1.ohdb_id.values,df1.ohdb_longitude.values, df1.ohdb_latitude.values):
    val0 = ds.sel(lon = lon0, lat = lat0, method = 'nearest').koppen.values
    if np.isnan(val0):
        print(ohdb_id0)
    df0 = df.loc[df.koppen_code==val0,:].reset_index(drop=True)
    df0['ohdb_id'] = ohdb_id0
    koppen.append(df0)
koppen = pd.concat(koppen)
df1 = df1.merge(koppen, on = 'ohdb_id')
df1.to_csv('../data/basin_attributes_koppen.csv', index = False)

OHDB_007009994
OHDB_014000155
OHDB_012001727
OHDB_015000013
OHDB_007009344
OHDB_008006375
OHDB_007010152
OHDB_008001457
OHDB_008001517
OHDB_008006308
OHDB_008006399


In [38]:
fnames1 = glob.glob('../data_mswx/GRIT_catch_ave_std*1981*')
fnames2 = glob.glob('../data_mswx/*snow*1981*csv')
df1 = []
for fname in fnames1:
    df = pd.read_csv(fname)
    df = df.loc[df.stat=='ave',:].drop(columns=['stat'])
    df = df.melt(id_vars = 'time')
    df = df.rename(columns={'variable':'ohdb_id','value':fname.split('_')[-2]})
    df = df.set_index(['ohdb_id','time'])
    df1.append(df)
df1 = pd.concat(df1, axis = 1)
df2 = []
for fname in fnames2:
    df = pd.read_csv(fname)
    df = df.melt(id_vars = 'time').rename(columns={'variable':'ohdb_id','value':fname.split('_')[-2]})
    df = df.set_index(['ohdb_id','time'])
    df2.append(df)
df2 = pd.concat(df2, axis = 1)
df = pd.concat([df1, df2], axis = 1)
df

P       Tmax       Tmin         SWd  \
ohdb_id        time                                                     
OHDB_009000845 1981-01-01  4.960733  21.503464  14.394141  197.006973   
               1981-01-02  0.996564  21.936342  13.685956  261.358429   
               1981-01-03  0.942824  23.212969  15.353713  257.343262   
               1981-01-04  1.474520  23.235859  14.299811  253.723099   
               1981-01-05  2.216167  22.546692  14.769879  264.795990   
...                             ...        ...        ...         ...   
OHDB_004000197 1981-12-27  0.000000  37.006573  23.601622  356.596588   
               1981-12-28  0.000000  37.264484  26.438501  333.311096   
               1981-12-29  1.523697  34.614761  26.281151  305.810211   
               1981-12-30  0.097780  34.238354  24.772182  290.525543   
               1981-12-31  0.085665  34.765869  23.399099  320.861359   

                                    Pres     RelHum   SpecHum         LWd  \
ohdb_id        time                                                         
OHDB_009000845 1981-01-01   82042.835938  89.206520  0.013872  362.504364   
               1981-01-02   82063.757812  86.056458  0.013364  351.786194   
               1981-01-03   81964.789062  83.134964  0.013904  357.235016   
               1981-01-04   81921.781250  80.910568  0.013259  357.349884   
               1981-01-05   81981.890625  81.769661  0.013302  357.425964   
...                                  ...        ...       ...         ...   
OHDB_004000197 1981-12-27   99503.234375  62.263279  0.015858  391.136322   
               1981-12-28   99575.695312  68.274765  0.018731  410.273987   
               1981-12-29   99908.812500  72.394539  0.019430  425.678711   
               1981-12-30  100040.312500  68.747940  0.017634  414.837036   
               1981-12-31  100057.140625  67.658249  0.016210  401.011902   

                               Wind  snowfall  snow  snowmelt  
ohdb_id        time                                            
OHDB_009000845 1981-01-01  1.267897       0.0   0.0       0.0  
               1981-01-02  1.476641       0.0   0.0       0.0  
               1981-01-03  1.403453       0.0   0.0       0.0  
               1981-01-04  1.580318       0.0   0.0       0.0  
               1981-01-05  1.587152       0.0   0.0       0.0  
...                             ...       ...   ...       ...  
OHDB_004000197 1981-12-27  2.072009       0.0   0.0       0.0  
               1981-12-28  3.301222       0.0   0.0       0.0  
               1981-12-29  3.667662       0.0   0.0       0.0  
               1981-12-30  3.042900       0.0   0.0       0.0  
               1981-12-31  2.774102       0.0   0.0       0.0  

[3911705 rows x 12 columns]

In [39]:
df = df.reset_index().rename(columns={'time':'date'})
df.groupby('ohdb_id').p_apply(lambda x:x.drop(columns=['ohdb_id']).to_csv(f'../data_mswx/mswx_each_basin/meteo_{x.ohdb_id.values[0]}_1981.csv',index=False))

<LAMBDA> DONE:   0%|          | 0/10717 [00:00<?, ?it/s]

""


In [41]:
def func(ohdb_id):
    df = pd.read_csv(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv')
    df = df.drop(columns=['p_mean','p_std'])
    df1 = pd.read_csv(f'../data_mswx/mswx_each_basin/meteo_{ohdb_id}_1981.csv')
    df1.columns = [a.lower() for a in df1.columns]
    out = pd.concat([df, df1])
    df2 = pd.read_csv(f'../data_mswx/mswx_each_basin/{ohdb_id}_mswep_gleam.csv')
    out = out.merge(df2, on = 'date')
    out.to_csv(f'../data_mswx/mswx_each_basin/{ohdb_id}_1981-2023.csv', index = False)
ohdb_ids = pd.read_csv('../data/basin_attributes.csv')
ohdb_ids = ohdb_ids.ohdb_id.values
import multiprocessing as mp
pool = mp.Pool(12)
s = pool.map(func, ohdb_ids)

In [65]:
df = pd.read_csv('../data/dis_OHDB_seasonal4_Qmin7_Qmax7_1982-2023.csv')
df.ohdb_id.unique().shape

(10586,)

In [ ]:
!python plot_basin_attr.py